### Load data

In [1]:
import numpy as np
import pandas as pd


def load_data(dataset, train = True):
    
    if train:
        df_Xtr = pd.read_csv(f'dataset/Xtr{dataset}.csv', index_col=0)   # read training X
        df_Ytr = pd.read_csv(f'dataset/Ytr{dataset}.csv')  # read training y
        
        X = np.array(df_Xtr).squeeze()
        Y = df_Ytr.Bound.values.ravel().astype(float)
        
    else:
        df_Xte = pd.read_csv(f'dataset/Xte{dataset}.csv', index_col=0)   # read test X
        X = np.array(df_Xte).squeeze()
        Y = None
    
    return X, Y 

### Construct dictionary and the gram matrix

In [2]:
def dic_constr(X, k):
    Subseqs_dic ={}
    
    for idx, fulseq in enumerate(X):
        # compute all its subsequences
        subseqs = [ fulseq[i:i + k]  for i in range(len(fulseq) - k + 1)  ]
        
        for subseq in subseqs:
            # creat a new dict for new subsequences
            if not subseq in Subseqs_dic:
                Subseqs_dic[subseq] = {}
            if not str(idx) in Subseqs_dic[subseq]:
                Subseqs_dic[subseq][str(idx)] = 0
            
            Subseqs_dic[subseq][str(idx)] += 1
                    
    return Subseqs_dic

In [3]:
def gram_matrix(dicS, dicT, nrow, ncol):
    # dicS : always the training dictionary
    # dicT : test dictionary (could be the training set)
    # nrow : size of test set (could be the training set)
    # ncol : always size of training set
    import time
    gram_mat = np.zeros((nrow,ncol))
    vetS = np.zeros(ncol)
    vetT = np.zeros(nrow)
    
    start = time.time()
    for subseqT, subdicT in dicT.items():
        if subseqT in dicS.keys():
            subdicS = dicS[subseqT]
            
            for i, numi in subdicT.items():
                for j, numj in subdicS.items():
                    gram_mat[int(i),int(j)] += numi * numj   
#                     vetS[int(j)] += numj**2
#                     vetT[int(i)] += numi**2

    for subseqT, subdicT in dicT.items():
        for i, numi in subdicT.items():
            vetT[int(i)] += numi**2
    for subseqS, subdicS in dicS.items():
        for j, numj in subdicS.items():
            vetS[int(j)] += numj**2
            
                    
    denom = np.sqrt(np.outer(vetT,vetS))
#    denom[denom == 0] = 1  # should not have 0 by the definition
    gram_mat = np.divide(gram_mat,denom)
    
    stop = time.time()
    print('\t time {:.4f}'.format(stop - start))
    
    return gram_mat

### SVM solution

In [4]:
def sign(y):
    return 2*y-1

def binary(y):
    return ((y + 1) / 2).astype(int)

In [5]:
# SVM solution
from cvxopt import solvers, matrix

def SVM(K, y, lbd=1):
#     print(lbd)
    y = sign(y)  # {0,1} to {-1,1}
    n = len(y)
    
    q = - 2 * y
    P = K
    G = np.zeros((2*n, n))
    G[:n, :] = - np.diag(y)
    G[n:, :] = np.diag(y)
    h = np.zeros(2 * n)
    h[n:] = 1 / (2 * lbd * n)
    
    P = matrix(P)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)

    solvers.options['show_progress'] = False
    alpha = solvers.qp(P, q, G, h)

    return alpha

def predict(K, alpha):
    return binary(np.sign(K@alpha))

### Apply SVM on data and do the prediction

In [ ]:
paras = {'0': [8 ,0.0001], '1':[6,0.01], '2':[8,0.0001]}

dataset = ['0','1','2']
pre_y = []
for set in dataset:
    Xtr, Ytr = load_data(set,train =True)
    Xte,_ = load_data(set,train = False)

    # length of subsequence
    k = paras[set][0]
    lbd = paras[set][1]
    
    dic_tr = dic_constr(Xtr, k)
    dic_te = dic_constr(Xte, k)

    len_tr, len_te = len(Xtr), len(Xte)

    # the gram matrix 
    print('Compute the training gram matrix...')
    gram_mat_train = gram_matrix(dic_tr,dic_tr, len_tr,len_tr)
    print('Compute the testing gram matrix...')
    gram_mat_test = gram_matrix(dic_tr, dic_te, len_te,len_tr)

    # SVM solution
    sol = SVM(gram_mat_train, Ytr,lbd)
    pred_train = predict(gram_mat_train, sol['x'])
    acc_train = np.sum(np.abs(pred_train.squeeze() - Ytr)) / len_tr
    acc_train = 1-acc_train
    print('Length of subsequence is : {:}, and the accuracy on training set is {:.4f}'.format(k, acc_train))

    # output on the test set
    pred_test = predict(gram_mat_test, sol['x'] ).squeeze()
    pre_y.append(pred_test)

Compute the training gram matrix...
	 time 1.9172
Compute the testing gram matrix...
	 time 0.9635
Length of subsequence is : 8, and the accuracy on training set is 0.9880
Compute the training gram matrix...


In [47]:
pred_all = np.array(pre_y).reshape(3000)
pred = pd.Series(pred_all.astype('int'), name="Bound")
pred.index = pd.Series(np.arange(len(pred_all)), name="Id")
pred.to_csv('Yte.csv')